# Text Processing

In [230]:
# import packages
import pandas as pd
import numpy as np

# Import NLTK
import nltk
nltk.download('wordnet')

spacy.prefer_gpu()


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/avielstern/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [534]:
# import june date with sentiment analysis
df = pd.read_csv('data/2020-08_sentiment.csv')

# 1. Setup text data for matrix transformation with TFIDF
1. Punctuation, lowercase
2. Stop Words
3. Lemmatization
4. Tokenization

In [535]:
# import packages
import re
import string
import spacy
spc = spacy.load("en_core_web_sm")
spacy.prefer_gpu()

# import stopwords from spacy
all_stopwords = nlp.Defaults.stop_words


#add words to stopword list
all_stopwords |= {"likes","like","ya", "-PRON-", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z", "—"}

print(string.punctuation)


from nltk.tokenize import TweetTokenizer


def my_tokenizer(tweet):

    for punctuation_mark in string.punctuation:
        # Remove punctuation and set to lower case
        tweet = tweet.replace(punctuation_mark,'').lower()

    tknzr = TweetTokenizer()
    words = tknzr.tokenize(tweet)

    lem_words = []

    for i in words:
        token = sp(i)
        for word in token:
            lem_words.append(word.lemma_)

    tokens_filtered= [word for word in lem_words if not word in all_stopwords]
    tweet = (" ").join(tokens_filtered)

    for punctuation_mark in string.punctuation:
        tweet = tweet.replace(punctuation_mark,'')

   
    tweet = re.sub(r'^\s+', '', tweet)  # remove spaces at the beginning
    tweet = re.sub(r'\s+$', '', tweet)  # remove spaces at the end
    tweet = re.sub(r'\s+', ' ', tweet)  # replace consecutive spaces
    
    # Retokenize again: 
    tknzr = TweetTokenizer()
    tokenized_tweet = tknzr.tokenize(tweet)
    
    return tokenized_tweet

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


# 2. TFIDF 
Convert a collection of raw documents to a matrix of TF-IDF features.

In [526]:
len(df['full_text'])

117852

In [531]:
# min_df must be at least 25% of the doc
tfidf = TfidfVectorizer(min_df=0.25, tokenizer=my_tokenizer, ngram_range = (1,3))

In [553]:
X = df['full_text']

In [554]:
tfidf = TfidfVectorizer(min_df=0.05, tokenizer=my_tokenizer, ngram_range = (1,3))

tfidf_vect = tfidf.fit_transform(X).todense()
tfidf_vect.shape

(117852, 41)

    min_df (float or int, default=1)

    When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float in range of [0.0, 1.0], the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

In [548]:
word_counts = np.array(np.sum(tfidf_vect, axis=0)).reshape((-1,))
words = np.array(tfidf.get_feature_names())
words_df = pd.DataFrame({"word":words, 
                         "count":word_counts})
words_df.sort_values(by="count", ascending=False).head(10)

,word,count
19,realdonaldtrump,18.285218
14,kamalaharris,16.712841
13,joebiden,14.178305
18,president,12.334887
17,people,11.241789
25,trump,7.891518
0,america,6.146243
26,vote,5.946646
22,stand,3.913132
15,life,3.738629


In [549]:
words_df.sort_values(by="count", ascending=False).head()

,word,count
19,realdonaldtrump,18.285218
14,kamalaharris,16.712841
13,joebiden,14.178305
18,president,12.334887
17,people,11.241789


In [ ]:
len(words)

In [551]:
df_vector = pd.DataFrame(columns=tfidf.get_feature_names(), data=tfidf_vect)
display(df_vector)

,america,america forever,america forever vote,defender,defender america,defender america forever,depend,folk,folk defender,folk defender america,...,realdonaldtrump,realdonaldtrump life,realdonaldtrump life depend,stand,stand folk,stand folk defender,trump,vote,vote realdonaldtrump,vote realdonaldtrump life
0,0.183557,0.20964,0.20964,0.20964,0.20964,0.20964,0.20964,0.20964,0.20964,0.20964,...,0.135917,0.20964,0.20964,0.199804,0.20964,0.20964,0.0,0.187273,0.20964,0.20964
1,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000,0.0,0.000000,0.00000,0.00000
2,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.579385,0.00000,0.00000,0.000000,0.00000,0.00000,0.0,0.000000,0.00000,0.00000
3,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000,0.0,0.000000,0.00000,0.00000
4,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000,0.0,0.000000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.473563,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.350655,0.00000,0.00000,0.515479,0.00000,0.00000,0.0,0.000000,0.00000,0.00000
96,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000,0.0,0.000000,0.00000,0.00000
97,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000,0.0,0.000000,0.00000,0.00000
98,0.738676,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000,0.0,0.000000,0.00000,0.00000


In [ ]:
df_vector.to_csv('vector_08_2020.csv')